## Glow GWAS QC Tutorial 

This documenent contains a replication of the tutorial pipeline in [A tutorial on conducting genome‐wide association studies:
Quality control and statistical analysis](https://www.ncbi.nlm.nih.gov/pubmed/29484742/) (Marees et al. 2017) using Spark and Glow.

The data for this tutorial comes from a HapMap (phase III) and consists of 1,457,897 SNPs for 165 individuals. The documentation for the publication above states that:

> This tutorial uses freely available HapMap data: hapmap3_r3_b36_fwd.consensus.qc. We simulated a binary outcome measure (i.e., a binary phenotypic trait) and added this to the dataset. The outcome measure was only simulated for the founders in the HapMap data. This data set will be referred to as HapMap_3_r3_1. The HapMap data, without our simulated outcome measure, can also be obtained from http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/2010-05_phaseIII/plink_format/ 

The dataset itself is very small.  File size by format:

- ```HapMap_3_r3_1.{bed, bim, fam}```: 98M 
- ```HapMap_3_r3_1.parquet```: 92M (snappy compressed)
- ```HapMap_3_r3_1.vcf```: 968M (uncompressed)
- ```HapMap_3_r3_1.mt```: 101M (hail MatrixTable)

#### Companion Analysis

See [gwas-tutorial-plink](gwas-tutorial-plink) for the ```plink``` commands used in the tutorial publication and their corresponding outputs.

#### Contents

- [Load Raw Data](#load_raw_data): Reading plink files (.bed, .bim, .fam)
    - **Wide vs Long Format**: Comparing simple operations with vcf-esque wide format to stacked, long format
    - **Phenotypes and Demographics**: Loading gender and outcome data (the outcome is simulated in this case but everything else is real)
- [Step 1: Sample/Variant Absence Filter](#step_1)
- [Step 2: Gender Discrepancy](#step_2)
- [Step 3: Autosomal Variants and MAF Filtering](#step_3)
- [Step 4: Hardy-Weinberg Equilibrium Filtering](#step_4)
- [Step 5: Heterozygosity Filtering](#step_5)

In [1]:
// Load samtools lib before glow as this is necessary to avoid this error on vcf writes:
// htsjdk.variant.variantcontext.VariantContextBuilder.getGenotypes() method not found
import $ivy.`com.github.samtools:htsjdk:2.21.1`
import $file.^.init.spark, spark._
import $file.^.init.paths, paths._
import $file.^.init.glow, glow._
import $file.^.init.benchmark, benchmark._
import $file.^.init.{plotly => init_plotly}, init_plotly._
import $file.^.init.{functions => glow_functions}, glow_functions._
import sys.process._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import io.projectglow.Glow
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond.{init => init_plotly_js, _}
import better.files.File
import org.apache.log4j.{Level, Logger}
Logger.getLogger("io.projectglow.plink").setLevel(Level.WARN)

def timeop[T](op: String)(block: => T) = optimer("glow", op, block)

val ss = getLocalSparkSession()
import ss.implicits._
Glow.register(ss)

init_plotly_js(offline=false)

val data_dir = GWAS_TUTORIAL_DATA_DIR / "1_QC_GWAS"

Loading spark-stubs


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/01/16 04:16:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $ivy.$                                  

import $file.$           , spark._

import $file.$           , paths._

import $file.$          , glow._

import $file.$               , benchmark._

import $file.$                             , init_plotly._

import sys.process._

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.functions._

import io.projectglow.Glow

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond.{init => init_plotly_js, _}

import better.files.File

import org.apache.log4j.{Level, Logger}

defined function timeop
ss: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@631e51e9
import ss.implicits._

data_dir: File = /home/eczech/data/gwas/tutorial/1_QC_GWAS

In [2]:
// Helper function for fetching materialized datasets
def getOrCreate(path: File, fn: => DataFrame): DataFrame = {
    if (!path.exists)
        fn.write.parquet(path.toString)
    ss.read.parquet(path.toString)
}

defined function getOrCreate

<h2><a id="load_raw_data">Load Raw Data</a></h2>

The data for this tutorial is stored according to the ```plink``` specification discussed, in detail, in the [publication](https://www.ncbi.nlm.nih.gov/pubmed/29484742/) section entitled "2.1 Data Format".  Fortunately, there is a [plink](https://glow.readthedocs.io/en/latest/etl/variant-data.html#plink) input format provided within Glow that can be used to load the data.

In [3]:
// Immediately load and cache the plink dataset as parquet
val df = getOrCreate(
    File(data_dir / QC0_FILE + ".parquet"),
    ss.read.format("plink").load(data_dir / QC0_FILE + ".bed" toString)
)
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [4]:
df.drop("genotypes").show(3, false)

+----------+------------+--------+---------+---------+---------------+----------------+
|contigName|names       |position|start    |end      |referenceAllele|alternateAlleles|
+----------+------------+--------+---------+---------+---------------+----------------+
|6         |[rs1890312] |0.0     |153585564|153585565|G              |[A]             |
|6         |[rs317105]  |0.0     |153585687|153585688|G              |[A]             |
|6         |[rs10499272]|0.0     |153585724|153585725|G              |[C]             |
+----------+------------+--------+---------+---------+---------------+----------------+
only showing top 3 rows



In [5]:
df
    .selectExpr("element_at(names, 1) as variant", "explode(genotypes) as genotypes")
    .select("variant", "genotypes.*")
    .show(3, false)

+---------+------------+------+
|variant  |sampleId    |calls |
+---------+------------+------+
|rs1890312|1328_NA06989|[0, 0]|
|rs1890312|1377_NA11891|[0, 0]|
|rs1890312|1349_NA11843|[0, 0]|
+---------+------------+------+
only showing top 3 rows



Check to see how often the ```names``` and ```alternateAlleles``` fields have more or less than one value:

In [6]:
df.select(size($"names"), size($"alternateAlleles")).dropDuplicates().show

+-----------+----------------------+
|size(names)|size(alternateAlleles)|
+-----------+----------------------+
|          1|                     1|
+-----------+----------------------+



Show allele encoding frequencies:

In [7]:
// The allele encodings include "D"/"I", which will be problematic later on
df.groupBy($"referenceAllele", $"alternateAlleles"(0)).count.show

+---------------+-------------------+------+
|referenceAllele|alternateAlleles[0]| count|
+---------------+-------------------+------+
|              G|                  A|291730|
|              G|                  C| 34388|
|              C|                  T|292773|
|              T|                  G| 60109|
|              A|                  G|252469|
|              C|                  G| 34331|
|              C|                  A| 66008|
|              T|                  C|252457|
|              A|                  C| 60036|
|              G|                  T| 65637|
|              T|                  A| 23748|
|              A|                  T| 23868|
|              I|                  D|   226|
|              D|                  I|   117|
+---------------+-------------------+------+



Show that dataset is dense (i.e. ```genotypes``` array has constant size):

In [8]:
df.groupBy(size($"genotypes")).count.show

+---------------+-------+
|size(genotypes)|  count|
+---------------+-------+
|            165|1457897|
+---------------+-------+



Pull a count of variants and samples/individuals:

In [10]:
timeop("qc0-count") {
    println(count(df))
}

(1457897,165)
Elapsed time: 0.3 seconds


In [11]:
timeop("qc0-count-unique") {
    println(uniqueCount(df))
}

(1457897,165)
Elapsed time: 19.2 seconds


### Wide vs Long Format

Note that counting unique samples and variants was actually quite slow.  This takes around 20 seconds in local mode yet a similar ```plink``` operation that fetches this information takes less than 1 second.  One possible reason for this could be that wide data formats (i.e. a large array in each row) are more difficult to optimize than long formats.  The code below will stack the raw plink dataset to produce a row for each variant and sample combination (rather than data for all samples being in each per-variant row) and see if a simple counting operation like this runs more quickly.

**note**: the "_lf" (aka long format) suffix will be applied to any data frame in this document as a convention indicating the change in structure

In [12]:
// Create the "long format" table by exploding the genotype calls into their own rows
def df_lf = df.withColumn("genotypes", explode($"genotypes"))

defined function df_lf

In [13]:
// Run the same counting operation
timeop("qc0-count-unique-lf") {
    df_lf.agg(
        countDistinct("genotypes.sampleId").as("numSamples"), 
        countDistinct($"names"(0)).as("numVariants")
    ).show
}

+----------+-----------+
|numSamples|numVariants|
+----------+-----------+
|       165|    1457897|
+----------+-----------+

Elapsed time: 23.6 seconds


Counting on top of a materialized dataset in long format was actually slower in this case (about ~21s instead of 17s).  The tutorial dataset only consists of ~1.5M SNPs and 165 people so it may be that the opposite becomes true for a sufficiently large experiment.  The plink command below shows how similar information can be retrieved for comparison.

In [14]:
timeop("qc0-count-plink") {
    (s"/usr/local/plink/plink --bfile ${data_dir}/HapMap_3_r3_1 --freq counts" #| "grep people\\|variants").!!
}

treat these as missing.


Elapsed time: 0.4 seconds


res13: String = """1457897 variants loaded from .bim file.
165 people (80 males, 85 females) loaded from .fam.
"""

### Phenotypes and Demographics

The gender and phenotype information for this data is stored in a separate .fam file, which per the [plink specification](https://www.cog-genomics.org/plink/1.9/formats#fam) is a headerless, space-delimited csv file.

A less obvious but key feature of this dataset includes the ability to identify "founders" in a GWAS population.  A population founder is defined generally as one of a small number of individuals isolated from a larger population that spawn a new but bottlenecked population.  This could be defined more specifically in a number of ways but in the context of ```plink```, this has a very specific definition (see [here](http://zzz.bwh.harvard.edu/plink/thresh.shtml#maf)) stating that founders are:

> individuals for whom the paternal and maternal individual codes and both 0.

The ```iidp``` and ```iidm``` codes are either "0" for no parent or equal to the ```iid``` of another individual in the dataset.  Founders are then identifiable where both of these values are "0".

In [15]:
def dp = ss.read.option("delimiter", " ")
    .csv(data_dir / QC0_FILE + ".fam" toString)
    .toDF("fid","iid","iidp","iidm", "gender", "phenotype")
    .withColumn("sampleId", concat($"fid", lit("_"), $"iid"))
dp.show(3)

+----+-------+----+----+------+---------+------------+
| fid|    iid|iidp|iidm|gender|phenotype|    sampleId|
+----+-------+----+----+------+---------+------------+
|1328|NA06989|   0|   0|     2|        2|1328_NA06989|
|1377|NA11891|   0|   0|     1|        2|1377_NA11891|
|1349|NA11843|   0|   0|     1|        1|1349_NA11843|
+----+-------+----+----+------+---------+------------+
only showing top 3 rows



defined function dp

In [16]:
// Note that phenotypes can be missing (-9) but gender is always present in this case
dp.groupBy("gender", "phenotype").count.sort($"gender", $"phenotype").show

+------+---------+-----+
|gender|phenotype|count|
+------+---------+-----+
|     1|       -9|   23|
|     1|        1|   27|
|     1|        2|   30|
|     2|       -9|   30|
|     2|        1|   29|
|     2|        2|   26|
+------+---------+-----+



In [17]:
dp
    .withColumn("has_father_in_dataset", $"iidp" =!= "0")
    .withColumn("has_mother_in_dataset", $"iidm" =!= "0")
    .withColumn("is_founder", !$"has_father_in_dataset" && !$"has_mother_in_dataset")
    .groupBy("has_father_in_dataset", "has_mother_in_dataset", "is_founder")
    .count.show

+---------------------+---------------------+----------+-----+
|has_father_in_dataset|has_mother_in_dataset|is_founder|count|
+---------------------+---------------------+----------+-----+
|                false|                false|      true|  112|
|                 true|                 true|     false|   48|
|                false|                 true|     false|    3|
|                 true|                false|     false|    2|
+---------------------+---------------------+----------+-----+



## QC

<h2><a id="step_1">Step 1: Sample/Variant Absence Filter</a></h2>

This section covers the tutorial steps related to filtering variants and samples based on their missingness in the data (aka call rate).  These steps are defined as:

- Delete SNPs with missingness >0.2.
- Delete individuals with missingness >0.2.
- Delete SNPs with missingness >0.02.
- Delete individuals with missingness >0.02 
  
In Glow, [Sample Quality Control](https://glow.readthedocs.io/en/latest/etl/sample-qc.html) functions produce summary statistics that are useful for sample (i.e. individual) QC, including call rates.  The function ```sample_call_summary_stats``` is an aggregation that when applied to an entire data frame will produce a single row with a big array containing all the data for each sample.  See [SampleCallSummaryStats.scala](https://github.com/projectglow/glow/blob/75ea111e0ba86001ea213df103373125b8732778/core/src/main/scala/io/projectglow/sql/expressions/SampleCallSummaryStats.scala#L42) for implementation details.

Similarly, [Variant Quality Control](https://glow.readthedocs.io/en/latest/etl/variant-qc.html) functions produce statistics across samples for individual variants.  The ```call_summary_stats``` function is a transformation that will produce statistics for each row (since the plink/vcf convention is to have one variant per row with all the samples in columns, this is possible as a per-row transformation). See [VariantQcExprs.scala](https://github.com/projectglow/glow/blob/master/core/src/main/scala/io/projectglow/sql/expressions/VariantQcExprs.scala#L168) for implementation details.

In [18]:
// In each step, *materialized* datasets will be loaded to base all operations on for that
// step to ensure that execution time comparisons are compatible (otherwise they are
// dependent on the performance of previous steps)
val df = ss.read.parquet(data_dir / QC0_FILE + ".parquet" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

#### Sample Filtering

Use the built-in glow function to get per-sample QC metrics:

In [19]:
def df_stat = df.selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
df_stat.printSchema

root
 |-- qc: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- callRate: double (nullable = true)
 |    |    |-- nCalled: long (nullable = true)
 |    |    |-- nUncalled: long (nullable = true)
 |    |    |-- nHomRef: long (nullable = true)
 |    |    |-- nHet: long (nullable = true)
 |    |    |-- nHomVar: long (nullable = true)
 |    |    |-- nSnp: long (nullable = true)
 |    |    |-- nInsertion: long (nullable = true)
 |    |    |-- nDeletion: long (nullable = true)
 |    |    |-- nTransition: long (nullable = true)
 |    |    |-- nTransversion: long (nullable = true)
 |    |    |-- nSpanningDeletion: long (nullable = true)
 |    |    |-- rTiTv: double (nullable = true)
 |    |    |-- rInsertionDeletion: double (nullable = true)
 |    |    |-- rHetHomVar: double (nullable = true)



defined function df_stat

Show that this is a single row:

In [20]:
df_stat.count

res19: Long = 1L

Show that exploding this array gives a number of rows equal to expected number of samples:

In [21]:
df_stat.selectExpr("explode(qc)").count

res20: Long = 165L

Show what the call rate distribution looks like across all samples:

In [22]:
df_stat
    .selectExpr("explode(qc) as qc")
    .fn(d => {
        Histogram(
            x=d.select("qc.callRate").collect.map(_.getAs[Double]("callRate")).toList
        ).plot(title="Sample Call Rate Distribution", xaxis=Axis(title="Call Rate"))
    })

res21: String = "plot-f454ab2e-2f2e-48af-a55a-a4f38ee1c686"

Now that there is a way to get the per-sample statistics, we need a way to apply a filtering on this data back to the original dataset.  The current documentation which includes a [notebook](https://glow.readthedocs.io/en/latest/_static/notebooks/etl/sample-qc-demo.html) for doing this appears to be out of date (it suggests using a form of output from ```sample_call_summary_stats``` that no longer exists) but a slight adapatation of this below will work.

In [23]:
// Filter a genotype dataset to samples with a minimum call rate (across variants)
def filterBySampleCallRate(threshold: Double)(df: DataFrame): DataFrame = { 
    df
        // Cross join original dataset with single-row data frame containing a map like (sampleId -> QC stats)
        .crossJoin(
            df
            .selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
            .selectExpr("map_from_arrays(qc.sampleId, qc) as qc")
        )
        // For each row, filter the genotypes array (which has one element per sampleId) based on QC map lookup
        .selectExpr("*", s"filter(genotypes, g -> qc[g.sampleId].callRate >= ${threshold}) as filtered_genotypes")
        // Remove intermediate fields 
        .drop("qc", "genotypes").withColumnRenamed("filtered_genotypes", "genotypes")
        // Ensure that the original dataset schema was preserved
        .transform(d => {assert(d.schema.equals(df.schema)); d})
}

defined function filterBySampleCallRate

#### Variant Filtering

In [24]:
def df_stat = df.selectExpr("call_summary_stats(genotypes) as qc")
df_stat.printSchema

root
 |-- qc: struct (nullable = true)
 |    |-- callRate: double (nullable = false)
 |    |-- nCalled: integer (nullable = false)
 |    |-- nUncalled: integer (nullable = false)
 |    |-- nHet: integer (nullable = false)
 |    |-- nHomozygous: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- nNonRef: integer (nullable = false)
 |    |-- nAllelesCalled: integer (nullable = false)
 |    |-- alleleCounts: array (nullable = true)
 |    |    |-- element: integer (containsNull = false)
 |    |-- alleleFrequencies: array (nullable = true)
 |    |    |-- element: double (containsNull = false)



defined function df_stat

Show that the number of QC results in this case equals the number of variants:

In [25]:
df_stat.count

res24: Long = 1457897L

In [26]:
// Technical note: do histogram binning manually with a large number of elements 
// because plotly-scala will put data provided as json object in cell output
df_stat
    .withColumn("bin", bround($"qc.callRate"/.005)*.005)
    .groupBy("bin").count.sort($"bin".asc)
    .fn(d => {
        Bar(
            x=d.map(_.getAs[Double]("bin")).collect.toList,
            y=d.map(_.getAs[Long]("count")).collect.toList
        ).plot(
            title="Variant Call Rate Distribution", 
            xaxis=Axis(title="Call Rate"),
            yaxis=Axis(`type`=AxisType.Log, title="Num Variants")
        )
    })

res25: String = "plot-1cd8a685-96b7-4d73-ab70-e022b6d65d6b"

In [27]:
// Filter a genotype dataset to variants with a minimum call rate (across samples)
def filterByVariantCallRate(threshold: Double)(df: DataFrame): DataFrame = { 
    df
        .selectExpr("*", "call_summary_stats(genotypes) as qc")
        .filter($"qc.callRate" >= threshold)
        .drop("qc")
        .transform(d => {assert(d.schema.equals(df.schema)); d})
}

defined function filterByVariantCallRate

#### Combined Filtering

Use the functions defined above to create a filtering operation that replicates that in the tutorial.

In [28]:
// Invert the .2 and .02 thresholds below but preserve order of filter application
val df_qc1 = df
    .transform(filterByVariantCallRate(threshold=.8))
    .transform(filterBySampleCallRate(threshold=.8))
    .transform(filterByVariantCallRate(threshold=.98))
    .transform(filterBySampleCallRate(threshold=.98))

val ct = timeop("qc1") {
    count(df_qc1)
}

Elapsed time: 257.3 seconds


df_qc1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [contigName: string, names: array<string> ... 6 more fields]
ct: (Long, Int) = (1430443L, 165)

Validate results against tutorial data at this stage:

In [29]:
assert(count(ss.read.format("plink").load(data_dir / QC1_FILE + ".bed" toString)) == ct)

Materialize results for following step:

In [30]:
df_qc1.write.mode("overwrite").parquet(data_dir / QC1_FILE + ".parquet" toString)

<h2><a id="step_2">Step 2: Gender Discrepancy</a></h2>

The ```plink``` "--check-sex" command uses homozygosity rates on sex chromosomes to determine if the gender assigned to a person (in the ```.fam``` file) is unlikely.  The tutorial documentation states that an "F value" is called and that:

> This F value is based on the X chromosome inbreeding (homozygosity) estimate.

The [plink check-sex documentation](https://www.cog-genomics.org/plink/1.9/basic_stats#check_sex) is similarly vague in its description of this statistic:

> --check-sex normally compares sex assignments in the input dataset with those imputed from X chromosome inbreeding coefficients

It's not entirely clear what this statistic calculation requires, but the [source code](https://github.com/chrchang/plink-ng/blob/b6bb12d5a14523f53865843104aaa20ed057aff5/1.9/plink_misc.c#L3206) for it looks as though the calculation is close to simply being the homozygosity rate.  The [plink inbreeding coefficient docs](https://www.cog-genomics.org/plink/1.9/basic_stats#ibc) (which the ```--check-sex``` command statistic "is based on") may give some more details on exactly what the statistic is (it is lifted from GCTA) but for the purposes of this analysis, homozygosity rate will be used because we know which samples should be omitted and can validate against the ```plink``` results.

In [31]:
val df = ss.read.parquet(data_dir / QC1_FILE + ".parquet" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [32]:
// Show variant counts by chromosome (i.e. "contigName")
df.groupBy("contigName").count.sort($"contigName".cast("int").asc).show(25)

+----------+------+
|contigName| count|
+----------+------+
|         1|117459|
|         2|117501|
|         3| 97341|
|         4| 86388|
|         5| 88819|
|         6| 92037|
|         7| 75919|
|         8| 75777|
|         9| 64180|
|        10| 74379|
|        11| 71812|
|        12| 68941|
|        13| 52335|
|        14| 45626|
|        15| 42284|
|        16| 45090|
|        17| 38729|
|        18| 41141|
|        19| 26437|
|        20| 36624|
|        21| 19415|
|        22| 20310|
|        23| 31490|
|        25|   409|
+----------+------+



Homozygosity rates on X chromosomes for men are expected to be near 1 because there should be no second X chromsome for which alternate alleles can exist (outside of the pseudoautosomal region).  An analysis of the HapMap Phase III data (the same data being used here) in [Inference of Unexpected Genetic Relatedness among Individuals in HapMap Phase III](https://www.cell.com/ajhg/fulltext/S0002-9297(10)00427-1#secd9962300e927) showed that women have a heterozygous/homozygous ratio on X of about .33 (see Figure 2A).  This means that in the histogram below, men should all cluster near one while women should cluster near 3/(3 + 1) = .75.  There is one clear outlier, which is also what ```plink``` found (see the companion analysis).

In [33]:
def df_gender_impute = df
    // Filter to X/Y chromosome
    .filter($"contigName" === "23")
    // Get per-sample call statistics
    .selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
    .selectExpr("explode(qc) as qc")
    // Compute homozygosity rates for each person
    // See: https://github.com/projectglow/glow/blob/75ea111e0ba86001ea213df103373125b8732778/core/src/main/scala/io/projectglow/sql/expressions/SampleCallSummaryStats.scala#L162
    // for calculation
    .select($"qc.sampleId", (lit(1.0) - $"qc.nHet".cast("double") / $"qc.nCalled".cast("double")).as("homRate"))
    // Grab current gender assignment
    .join(
        dp.select($"sampleId", $"gender"),
        Seq("sampleId")
    )
    // Add imputed gender assignment
    .withColumn("imputedGender", when($"homRate" > .9 && $"gender" === "2", "1").otherwise($"gender"))

df_gender_impute.fn(d => {
        // Create histogram trace for each gender
        d.select("gender").dropDuplicates().map(_.getAs[String]("gender")).collect.map(g => 
            Histogram(
                x=d.filter($"gender" === g).map(_.getAs[Double]("homRate")).collect.toList, 
                // See https://www.cog-genomics.org/plink/1.9/formats#fam for encoding
                name=Map("1" -> "Male", "2" -> "Female")(g),
                xbins=Bins(0.7, 1.01, .01)
            )
        ).toSeq.plot(
            title="Sex Chromosome Homozygosity Rates",
            yaxis=Axis(`type`=AxisType.Log, title="Sample Count"),
            xaxis=Axis(title="Homozygosity Rate", range=(0.7, 1.01))
        )
    })

defined function df_gender_impute
res32_1: String = "plot-5f31b362-28c0-4e47-9546-400f80e2bff3"

Find the single female sample above the threshold:

In [34]:
df_gender_impute.filter($"gender" =!= $"imputedGender").show

+------------+------------------+------+-------------+
|    sampleId|           homRate|gender|imputedGender|
+------------+------------------+------+-------------+
|1349_NA10854|0.9982031397768111|     2|            1|
+------------+------------------+------+-------------+



Define the transformed version of the demographic data up through step 2:

In [35]:
// This only affects the pedigree data, not the genotype dataset
val dp_qc2 = timeop("qc2") {
    dp
        .join(df_gender_impute.select("sampleId", "imputedGender"), Seq("sampleId"), "left")
        .withColumn("gender", when($"gender" =!= $"imputedGender", $"imputedGender").otherwise($"gender"))
        .drop("imputedGender")
        .transform(d => {println(d.count); d})
        // While somewhat awkward here, this count of the genotype data is included to align
        // with what other tutorial do in this step
        .transform(d => {println(count(df)); d})
}
dp_qc2.show(3, false)

165
(1430443,165)
Elapsed time: 6.9 seconds
+------------+----+-------+----+----+------+---------+
|sampleId    |fid |iid    |iidp|iidm|gender|phenotype|
+------------+----+-------+----+----+------+---------+
|1328_NA06989|1328|NA06989|0   |0   |2     |2        |
|1377_NA11891|1377|NA11891|0   |0   |1     |2        |
|1349_NA11843|1349|NA11843|0   |0   |1     |1        |
+------------+----+-------+----+----+------+---------+
only showing top 3 rows



dp_qc2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sampleId: string, fid: string ... 5 more fields]

In [36]:
dp_qc2.groupBy("gender").count.withColumnRenamed("count", "new_count").join(
    dp.groupBy("gender").count.withColumnRenamed("count", "old_count"),
    Seq("gender")
).show

+------+---------+---------+
|gender|new_count|old_count|
+------+---------+---------+
|     2|       84|       85|
|     1|       81|       80|
+------+---------+---------+



In [37]:
dp_qc2.write.mode("overwrite").parquet(data_dir / QC2_FILE + ".fam.parquet" toString)

In [38]:
val dp_qc2 = ss.read.parquet(data_dir / QC2_FILE + ".fam.parquet" toString)

dp_qc2: DataFrame = [sampleId: string, fid: string ... 5 more fields]

<h2><a id="step_3">Step 3: Autosomal Variants and MAF Filtering</a></h2>

This step will remove variants for nonautosomal chromosomes as well SNPs that are too rare to be useful.

Regarding minor allele frequency (MAF) filtering, the tutorial states:

> The MAF threshold should depend on your sample size, larger samples can use lower MAF thresholds. Respectively, for large (N = 100.000) vs. moderate samples (N = 10000), 0.01 and 0.05 are commonly used as MAF threshold.

This dataset has N=165 (individuals) so the more aggressive 0.05 threshold will be used.

Additionally, ```plink``` defaults to calculating MAF only for population "founders".  The docs at [plink#maf](http://zzz.bwh.harvard.edu/plink/thresh.shtml#maf) state:

> This quantity is based only on founders (i.e. individuals for whom the paternal and maternal individual codes and both 0).

This can be disabled using the ```--nonfounders``` flag, which makes the filtering below equivalent to ignoring everything about samples entirely, but we will instead use the ```.fam``` file data to identify founders the same way.

In [39]:
val df = ss.read.parquet(data_dir / QC1_FILE + ".parquet" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [40]:
// Show MAF ignoring founder status
df
    .selectExpr("call_summary_stats(genotypes) as qc")
    .filter(size($"qc.alleleFrequencies") > 1)
    .withColumn("maf", array_min($"qc.alleleFrequencies"))
    .withColumn("bin", bround($"maf"/.01)*.01)
    .groupBy("bin").count.sort($"bin".asc)
    .fn(d => {
        Bar(
            x=d.map(_.getAs[Double]("bin")).collect.toList,
            y=d.map(_.getAs[Long]("count")).collect.toList
        ).plot(
            title="MAF Distribution",
            yaxis=Axis(`type`=AxisType.Log, title="Num Variants"),
            xaxis=Axis(title="Minor Allele Frequency (MAF)"),
            margin=Margin(b=150)
        )
    })

res39: String = "plot-505fa71c-f34d-4450-81e5-ff007ff8c7a2"

In [41]:
val df_qc3 = df
    // Limit to autosomal chromosomes
    .filter($"contigName".cast("int").between(1, 22))
    // Cross join to single-row dataset with founders map
    .crossJoin(
        dp_qc2
            .withColumn("isFounder", $"iidp" === "0" && $"iidm" === "0")
            .select("sampleId", "isFounder")
            .agg(map_from_arrays(collect_list($"sampleId"), collect_list($"isFounder")).as("isFounder"))
    )
    // Run call_summary_stats only for population founders
    .selectExpr("*", "call_summary_stats(filter(genotypes, g -> isFounder[g.sampleId])) as qc")
    // Ensure there are at least two alleles and if so, that MAF >= 5%
    .filter(size($"qc.alleleFrequencies") > 1 && array_min($"qc.alleleFrequencies") >= 0.05)
    // Remove intermediate fields
    .drop("qc", "isFounder")

val ct = timeop("qc3") {
    count(df_qc3)
}

Elapsed time: 95.0 seconds


df_qc3: DataFrame = [contigName: string, names: array<string> ... 6 more fields]
ct: (Long, Int) = (1073226L, 165)

In [42]:
assert(count(ss.read.format("plink").load(data_dir / QC3_FILE + ".bed" toString)) == ct)

In [43]:
df_qc3.write.mode("overwrite").parquet(data_dir / QC3_FILE + ".parquet" toString)

<h2><a id="step_4">Step 4: Hardy-Weinberg Equilibrium Filtering</a></h2>

This step will identify and eliminate SNPs that are not in Hardy-Weingberg equilibrium.  This happens in two phases
in the tutorial where in the first, SNPs for elimination are chosen using a less stringent p-value threshold (1e-6) based only on founders in the control group and in the second, SNPs across all individuals are chosen using a more stringent p-value threshold (1e-10).

The ```plink``` command for this is described at [plink#hwe](https://www.cog-genomics.org/plink/1.9/filter#hwe), which states that:

> Only founders are considered by this test; use --nonfounders to change this. Also, with case/control data, cases and missing phenotypes are normally ignored; override this with 'include-nonctrl'.

and:

> --hwe's 'midp' modifier applies the mid-p adjustment described in Graffelman J, Moreno V (2013) The mid p-value in exact tests for Hardy-Weinberg equilibrium. The mid-p adjustment tends to bring the null rejection rate in line with the nominal p-value, and also reduces the filter's tendency to favor retention of variants with missing data. We recommend its use.

The latter caveat is important because the Glow function ```hardy_weinberg``` (shown in [Variant QC](https://glow.readthedocs.io/en/latest/etl/variant-qc.html)) uses this implementation by default.  This can be seen in the source code for the [HWE function](https://github.com/projectglow/glow/blob/75ea111e0ba86001ea213df103373125b8732778/core/src/main/scala/io/projectglow/sql/expressions/VariantQcExprs.scala#L45) which uses [LeveneHaldane.scala](https://github.com/projectglow/glow/blob/75ea111e0ba86001ea213df103373125b8732778/core/src/main/scala/io/projectglow/sql/util/LeveneHaldane.scala#L26).  That implementation, take from Hail, is best documented in [hail/docs/misc/LeveneHaldane.pdf](https://github.com/hail-is/hail/blob/master/hail/python/hail/docs/misc/LeveneHaldane.pdf) and details the distribution used as part of the 2013 Graffelman J, Moreno V publication [The mid p-value in exact tests for Hardy-Weinberg equilibrium](https://www.ncbi.nlm.nih.gov/pubmed/23934608).

In [44]:
val df = ss.read.parquet(data_dir / QC3_FILE + ".parquet" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [45]:
df.select(expr("hardy_weinberg(genotypes) as hwe")).printSchema

root
 |-- hwe: struct (nullable = true)
 |    |-- hetFreqHwe: double (nullable = true)
 |    |-- pValueHwe: double (nullable = true)



In [46]:
df
    .select(expr("log10(hardy_weinberg(genotypes).pValueHwe)").as("p"))
    .withColumn("bin", bround($"p"/.1)*.1)
    .groupBy("bin").count.sort($"bin".asc)
    .fn(d => {
        Bar(
            x=d.map(_.getAs[Double]("bin")).collect.toList,
            y=d.map(_.getAs[Long]("count")).collect.toList
        ).plot(
            title="HWE P-Value Distribution",
            yaxis=Axis(`type`=AxisType.Log, title="Num Variants"),
            xaxis=Axis(title="Log10(p)")
        )
    })

res45: String = "plot-1f6fb909-95d5-4430-a9f7-d2a59f556eab"

In [47]:
val df_qc4 = df
    // Phase 1: controls + founders only, less stringent threshold
    .crossJoin(
        dp_qc2
            .withColumn("isFounder", $"iidp" === "0" && $"iidm" === "0")
            .withColumn("isControl", $"phenotype" === "1") // 1=control, 2=case, -9=missing
            .filter($"isFounder" && $"isControl")
            .agg(map_from_arrays(collect_list($"sampleId"), collect_list(lit(true))).as("isTarget"))
    )
    .withColumn("hwe", expr("hardy_weinberg(filter(genotypes, g -> coalesce(isTarget[g.sampleId], false)))"))
    .filter($"hwe.pValueHwe" > 1e-6)
    .drop("isTarget", "hwe")

    // Phase 2: all samples, more stringent threshold
    .withColumn("hwe", expr("hardy_weinberg(genotypes)"))
    .filter($"hwe.pValueHwe" > 1e-10)
    .drop("hwe")

val ct = timeop("qc4") {
    count(df_qc4)
}

Elapsed time: 30.7 seconds


df_qc4: DataFrame = [contigName: string, names: array<string> ... 6 more fields]
ct: (Long, Int) = (1073226L, 165)

In [49]:
assert(count(ss.read.format("plink").load(data_dir / QC4_FILE + ".bed" toString)) == ct)

In [50]:
df_qc4.write.mode("overwrite").parquet(data_dir / QC4_FILE + ".parquet" toString)

<h2><a id="step_5">Step 5: Heterozygosity Filtering</a></h2>

This step is simple in principle.  The objective is to identify and remove samples with an unusually large or small heterozygosity rate.  One difficulty that arises when doing this though is that estimating this rate is often very biased due to LD.  The most common approach for dealing with this issue is to select a single variant from a group of variants in high LD and estimate the rate using only these representative variants.  Another difficulty is that genomic regions with frequent inversion mutations result in very different LD relationships for SNPs near the breakpoints.  Both are detailed more below.

#### Variant Pruning/Clumping Primer

There are several approaches that may be taken for this kind of representative variant election and two common ones are known as "pruning" and "clumping".  Pruning is supported by both ```plink``` and Hail (via [ld_prune](https://hail.is/docs/0.2/_modules/hail/methods/statgen.html#ld_prune)).  This process works by ordering the variants along the genome, moving in windows of at most some base pair block size, determining the correlation between all variants in that window, and greedily removing any with a correlation above some threshold.  The (now deprecated) Hail 0.1 docs for ld_prune show a simplified version of their algorithm (the 0.2 [ld_prune](https://hail.is/docs/0.2/methods/genetics.html?highlight=ld_prune#hail.methods.ld_prune) algorithm is substantially more complicated):

```python
pruned_set = []
for v1 in contig:
    keep = True
    for v2 in pruned_set:
        if ((v1.position - v2.position) <= window and correlation(v1, v2) >= r2):
            keep = False
    if keep:
        pruned_set.append(v1)
```

In ```plink```, a command to do this often looks like ```plink --bfile my-data --indep-pairwise 50 5 0.2```.  The ```--indep-pairwise``` flag tells ```plink``` to "produce a pruned subset of markers that are in approximate linkage equilibrium with each other, writing the IDs to plink.prune.in (and the IDs of all excluded variants to plink.prune.out)" (see [plink#LD](https://www.cog-genomics.org/plink/1.9/ld#indep)).

Glow provides no utilities for performing operations like this.  Until this point in the tutorial, it has been fairly straightforward to find equivalent or near-equivalent Spark operators for what ```plink``` does but iterative algorithms utilizing both genomic distance and pairwise correlation are not easy to concisely replicate in this context.

The "clumping" approach, however, might constitute a method that is easier to replicate and this is [post](https://privefl.github.io/bigsnpr/articles/pruning-vs-clumping.html#clumping) by Florian Privé demonstrates the difference well.  This is a an example from the [bigsnpr](https://github.com/privefl/bigsnpr) R package ([c++ clumping implementation](https://github.com/privefl/bigsnpr/blob/321d45ec9c8e2c9d1ec5576ac279fefbec6e7159/src/clumping.cpp)) with an accompanying manuscript by Florian called [Efficient analysis of large-scale genome-wide data with two R packages: bigstatsr and bigsnpr](https://doi.org/10.1093/bioinformatics/bty185).  He describes the difference between pruning and clumping [here](https://www.biostars.org/p/343818/) as:

> 1. pruning: it uses the first SNP (in genome order) and computes the correlation with the following ones (e.g. 50). When it finds a large correlation, it removes one SNP from the correlated pair, keeping the one with the largest minor allele frequency (MAF), thus possibly removing the first SNP. Then it goes on with the next SNP (not yet removed). So, in some worst case scenario, this algorithm may in fact remove all SNPs of the genome (expect one).

> 2. clumping: it uses some statistic (usually p-value in the case of GWAS/PRS) to sort the SNPs by importance (e.g. keeping the most significant ones). It takes the first one (e.g. most significant SNP) and removes SNPs that are too correlated with this one in a window around it. As opposed to pruning, this procedure makes sure that this SNP is never removed, keeping at least one representative SNP by region of the genome. Then it goes on with the next most significant SNP that has not been removed yet. In the case of computing principal components, there is no p-value available, so I propose to use the MAF instead as the statistic to rank SNPs (in decreasing order). Using MAFs makes clumping very similar to pruning, but without any worst-case scenario.

One of these methods are often used as precursor steps to objectives like:

- Variant zygosity statistics (as in the current step)
- Genetic relatedness (e.g. Identity by descent (IBD))
  - The [plink docs for IBD](https://www.cog-genomics.org/plink/1.9/ibd) immediately caution that:
  > These calculations are not LD-aware. It is usually a good idea to perform some form of LD-based pruning before invoking them
- Using PCA to identify population substructure
  - This [notebook](https://privefl.github.io/bigsnpr/articles/how-to-PCA.html) shows to use pruning prior to PCA
- Building a polygenic risk score (PRS) model

Because "LD-aware" analysis is important and the variant election algorithm needed for it may need to have different priorities, it is worth noting that both approaches make it possible to use some measure of importance for election.  It is possible to do something like this in with ```plink``` as mentioned in this [issue](https://github.com/chrchang/plink-ng/issues/27) by overriding the MAF estimates used to choose between two correlated variants.  A similar custom score could be defined for sorting in the clumping approach too.  This would make it possible to either prioritize variants based on association with an outcome phenotype (not appropriate for PCA prior to running GWAS regression models) or to prioritize them based on being in coding regions, as a couple examples.

#### Inversion Mutations

When an inversion occurs, SNPs near the breakpoints that were physically close often loose linkage with one another (and gain it with new ones).  This is described in [Evidence for large inversion polymorphisms in the human genome from HapMap data](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1781354/#!po=16.0714) (Bansal et al. 2007):

> Consider a genomic region that is inverted (with respect to the reference sequence) in a majority of the chromosomes in a population, and assume that we have genotyped markers on either side of the two breakpoints. In such a scenario, we would expect to see unusually higher levels of long-range LD than would be expected between markers that are physically distant. Furthermore, one would also observe low LD between pairs of markers that are physically close according to the reference sequence. 

The GWAS tutorial publication addresses this by simply ignoring a few regions where inversions are known to occur frequently (on chromsomes 6 (HLA), 8, and 17).

#### Implementation

**Aside on Hail ld_prune**: I had hoped that perhaps the Hail implementation for LD pruning would be efficient by comparison to ```plink``` (before looking more at how to do it with Spark directly) and encountered [LD prune performance is unacceptable](https://github.com/hail-is/hail/issues/4506).  I was somewhat encouraged by the later PR [make ld_prune fast again](https://github.com/hail-is/hail/pull/5078) and because this didn't end in a very clear result I tried it myself on the tutorial data.  I found that ```plink``` takes about 2 seconds to do what takes ~4 minutes with ```hl.ld_prune```.

For now, this step of the workflow will at least show how to use the Glow [Pipe Transformer](https://glow.readthedocs.io/en/latest/tertiary/pipe-transformer.html) to run ```plink``` commands for pruning prior to removing variants with abnormal heterozygosity rates.  ```plink``` is not built for streaming operations though so there are some awkward accomodations that need to be made, all shown below.

In [51]:
val df = ss.read.parquet(data_dir / QC4_FILE + ".parquet" toString)

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [52]:
// Get list of variants to use for heterozygosity rate estimation
def variants_to_keep = df
    // LD pruning in plink is based only on founders so apply the same filtering here
    // see: https://www.cog-genomics.org/plink/1.9/ld#indep
    .crossJoin(
        dp_qc2
            .withColumn("isFounder", $"iidp" === "0" && $"iidm" === "0")
            .select("sampleId", "isFounder")
            .agg(map_from_arrays(collect_list($"sampleId"), collect_list($"isFounder")).as("isFounder"))
    )
    .withColumn("genotypes", expr("filter(genotypes, g -> coalesce(isFounder[g.sampleId], false))"))

    // Remove three genome regions with high-frequency inversion mutations
    // * equivalent to --exclude inversion.txt --range in tutorial
    .filter(!(
        ($"contigName".cast("int") === 6 && $"start".between(25500000, 33500000)) ||
        ($"contigName".cast("int") === 8 && $"start".between(8135000, 12000000)) ||
        ($"contigName".cast("int") === 17 && $"start".between(40900000, 45000000)) 
    ))

    // The raw data contains "I" and "D" alleles for insertion and deletion which will cause an error
    // using the Glow vcf writer because the samtools java lib used under the hood doesn't understand
    // these encodings.  For this operation sepcifically, the nature of the mutation doesn't matter
    // so they can be recoded as if they were for SNPs instead of indels.  The results don't come 
    // out drastically different w/o this --> 104134 variants are kept instead of the expected 104144
    .withColumn("referenceAllele", when($"referenceAllele".isin("I", "D"), lit("A"))
                .otherwise($"referenceAllele"))
    .withColumn("alternateAlleles", when($"alternateAlleles"(0).isin("I", "D"), array(lit("T")))
                .otherwise($"alternateAlleles"))

    // The LD prune algorithm in plink technically requires that all variants be present in the same run
    // which means the same result won't be produced if the computation is done in parallel on separate
    // partitions as if it is done on a single partition.  The variants removed probably change very little
    // if this is done concurrently, but since we're going for equality with the plink-only results, use 1 partition
    .repartition(1)

    // Rows in vcf must be sorted by chromosome or plink with throw the error
    // ".bim file has a split chromosome"
    .sortWithinPartitions($"contigName".cast("int").asc, $"start".asc, $"end".asc)
    
    // Write the plink data as VCF 
    // WARNING: This will drop all sample phenotype/demographic data which plink will then treat as missing
    //          so it is important to avoid operations that require this info
    // NOTE: plink-pipe = 
    // #!/bin/bash
    // prefix=plink-$(cat /proc/sys/kernel/random/uuid)
    // plink --vcf /dev/stdin "${@:2}" --out prefix > /dev/null
    // cat prefix.$1
    .fn(d => 
        Glow.transform("pipe", d, Map(
            // prune.in = variants to keep, prune.out = variants to remove (they are single-col csvs)
            "cmd" -> "[\"plink-pipe\", \"prune.in\", \"--indep-pairwise\", \"50\", \"5\", \"0.2\"]",
            "inputFormatter" -> "vcf",
            "outputFormatter" -> "text",
            "inVcfHeader" -> "infer"
        ))
    )
    .map(_.getAs[String]("text")).collect.toList

defined function variants_to_keep

In [53]:
// The plink tutorial results in the retention of 104,144 variants out of 1,457,897 total
val ct = variants_to_keep.size
assert(ct == 104144)
ct

20/01/16 04:38:42 INFO PipeTransformer: hlsUsage:[pipe,{"pipeCmdTool":"plink"}]
20/01/16 04:38:42 INFO VCFHeaderUtils$: Inferring header for VCF writer
20/01/16 04:38:42 INFO Piper$: Beginning pipe with cmd List(plink-pipe, prune.in, --indep-pairwise, 50, 5, 0.2)
20/01/16 04:39:36 INFO InternalRowToVariantContextConverter: Field StructField(position,DoubleType,true) is present in data schema but does not have a VCF representation
20/01/16 04:39:36 INFO InternalRowToVariantContextConverter: Field StructField(isFounder,MapType(StringType,BooleanType,true),true) is present in data schema but does not have a VCF representation
20/01/16 04:40:20 INFO VCFInputFormatter: Closing VCF input formatter


ct: Int = 104144
res52_2: Int = 104144

In [54]:
// Compare this distribution to the "Heterozygosity Distribution" section in the companion analysis
def df_het_rate = df
    // Restrict to only variants in linkage equilibrium
    .filter($"names"(0).isin(variants_to_keep:_*))
    // Limit to autosomal chromosomes as in https://www.cog-genomics.org/plink/1.9/basic_stats#ibc (--het)
    .filter($"contigName".cast("int").between(1, 22))

    // Compute stats across variants
    .selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
    .withColumn("qc", explode($"qc"))

    // Determine rate and outliers
    .withColumn("hetRate", $"qc.nHet".cast("double") / $"qc.nCalled".cast("double"))

    // Identify outliers
    .transform(d => {
        d.crossJoin(d.agg(mean($"hetRate").as("hetRateMean"), stddev($"hetRate").as("hetRateStd")))
        .withColumn("hetRateLow", $"hetRateMean" - lit(3) * $"hetRateStd")
        .withColumn("hetRateHigh", $"hetRateMean" + lit(3) * $"hetRateStd")
        .withColumn("status", when(
            $"hetRate".between($"hetRateLow", $"hetRateHigh"), lit("ok")
        ).otherwise(lit("outlier")))
    })

df_het_rate
    .withColumn("bin", bround($"hetRate"/.001)*.001)
    .groupBy("status", "bin").count.sort($"status".asc, $"bin".asc)
    .fn(d => {
        Seq("ok", "outlier").map(s =>
            Bar(
                x=d.filter($"status" === s).map(_.getAs[Double]("bin")).collect.toList,
                y=d.filter($"status" === s).map(_.getAs[Long]("count")).collect.toList,
                name=s
            )
        ).plot(
            title="Sample Heterozygosity Rate Distribution",
            yaxis=Axis(title="Num Samples"),
            xaxis=Axis(title="Heterozygosity Rate")
        )
    })

20/01/16 04:44:48 INFO PipeTransformer: hlsUsage:[pipe,{"pipeCmdTool":"plink"}]
20/01/16 04:44:48 INFO VCFHeaderUtils$: Inferring header for VCF writer
20/01/16 04:44:48 INFO Piper$: Beginning pipe with cmd List(plink-pipe, prune.in, --indep-pairwise, 50, 5, 0.2)
20/01/16 04:45:40 INFO InternalRowToVariantContextConverter: Field StructField(position,DoubleType,true) is present in data schema but does not have a VCF representation
20/01/16 04:45:40 INFO InternalRowToVariantContextConverter: Field StructField(isFounder,MapType(StringType,BooleanType,true),true) is present in data schema but does not have a VCF representation
20/01/16 04:46:22 INFO VCFInputFormatter: Closing VCF input formatter


defined function df_het_rate
res53_1: String = "plot-57ea1b8c-6d5e-43eb-9b9c-8cc542e032e6"

In [55]:
val df_qc5 = df
    .crossJoin(
        df_het_rate
        .filter($"status" === "outlier")
        .agg(collect_set($"qc.sampleId").as("outliers"))
    )
    .withColumn("genotypes", expr("filter(genotypes, g -> !array_contains(outliers, g.sampleId))"))

val ct = timeop("qc5") {
    count(df_qc5)
}

20/01/16 04:48:27 INFO PipeTransformer: hlsUsage:[pipe,{"pipeCmdTool":"plink"}]
20/01/16 04:48:27 INFO VCFHeaderUtils$: Inferring header for VCF writer
20/01/16 04:48:27 INFO Piper$: Beginning pipe with cmd List(plink-pipe, prune.in, --indep-pairwise, 50, 5, 0.2)
20/01/16 04:49:20 INFO InternalRowToVariantContextConverter: Field StructField(position,DoubleType,true) is present in data schema but does not have a VCF representation
20/01/16 04:49:20 INFO InternalRowToVariantContextConverter: Field StructField(isFounder,MapType(StringType,BooleanType,true),true) is present in data schema but does not have a VCF representation
20/01/16 04:50:03 INFO VCFInputFormatter: Closing VCF input formatter


Elapsed time: 42.2 seconds


df_qc5: DataFrame = [contigName: string, names: array<string> ... 7 more fields]
ct: (Long, Int) = (1073226L, 163)

In [56]:
assert(count(ss.read.format("plink").load(data_dir / QC5_FILE + ".bed" toString)) == ct)

In [57]:
df_qc5.write.mode("overwrite").parquet(data_dir / QC5_FILE + ".parquet" toString)